In [9]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import keras
import tensorflow as tf
import pandas as pd
import datetime
import keras.backend as K
K.clear_session()
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout


First let's import keras and some other libraries

In [10]:
 data = pd.read_csv("m_stroman_pitches_FullArsenal.csv")

let's load the pitches. above are 3045 pitches, that marcus stroman threw in sequence from 2019

In [11]:
df = pd.DataFrame(data)

pitches = df['pitch_type']
pitches = pd.get_dummies(pitches) #get a one hot encoding for pitches
p_enc = pitches.to_numpy()

del df['pitch_type'] #don't need pitch type anymore

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

#label encode stance right or left
df["stand"] = le.fit_transform(df["stand"])

#one hot encode previous type, ball/strike/or in play
ty = pd.get_dummies(df['prev_type'])
df = df.drop('prev_type',axis = 1)
df = df.join(ty)

#one hot encode previous pitch string
prevs = pd.get_dummies(df['prev_pitch'])
df = df.drop('prev_pitch',axis = 1)
# Join the encoded df
df = df.join(prevs)
variables = df.to_numpy()

In [13]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split( 
    variables, p_enc, test_size=0.2)

In [16]:
model = Sequential()
from keras.optimizers import SGD
model.add(Dense(12, activation = 'relu', input_dim = 16))
model.add(Dense(6, activation = 'softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(trainX, trainY, epochs = 100,
            batch_size = 64)

Epoch 1/100
2435/2435 [==============================] - 0s 69us/step - loss: 1.5474 - accuracy: 0.3503
Epoch 2/100
2435/2435 [==============================] - 0s 19us/step - loss: 1.3807 - accuracy: 0.4131
Epoch 3/100
2435/2435 [==============================] - 0s 27us/step - loss: 1.3569 - accuracy: 0.4156
Epoch 4/100
2435/2435 [==============================] - 0s 24us/step - loss: 1.3398 - accuracy: 0.4292
Epoch 5/100
2435/2435 [==============================] - 0s 20us/step - loss: 1.3332 - accuracy: 0.4382
Epoch 6/100
2435/2435 [==============================] - 0s 26us/step - loss: 1.3307 - accuracy: 0.4324
Epoch 7/100
2435/2435 [==============================] - 0s 24us/step - loss: 1.3288 - accuracy: 0.4251
Epoch 8/100
2435/2435 [==============================] - 0s 29us/step - loss: 1.3186 - accuracy: 0.4378
Epoch 9/100
2435/2435 [==============================] - 0s 31us/step - loss: 1.3145 - accuracy: 0.4407
Epoch 10/100
2435/2435 [==============================] - 0s 27u

2435/2435 [==============================] - 0s 31us/step - loss: 1.2525 - accuracy: 0.4587
Epoch 80/100
2435/2435 [==============================] - 0s 32us/step - loss: 1.2573 - accuracy: 0.4571
Epoch 81/100
2435/2435 [==============================] - 0s 31us/step - loss: 1.2538 - accuracy: 0.4616
Epoch 82/100
2435/2435 [==============================] - 0s 20us/step - loss: 1.2522 - accuracy: 0.4600
Epoch 83/100
2435/2435 [==============================] - 0s 29us/step - loss: 1.2557 - accuracy: 0.4550
Epoch 84/100
2435/2435 [==============================] - 0s 29us/step - loss: 1.2584 - accuracy: 0.4513
Epoch 85/100
2435/2435 [==============================] - 0s 29us/step - loss: 1.2499 - accuracy: 0.4587
Epoch 86/100
2435/2435 [==============================] - 0s 26us/step - loss: 1.2522 - accuracy: 0.4554
Epoch 87/100
2435/2435 [==============================] - 0s 21us/step - loss: 1.2484 - accuracy: 0.4608
Epoch 88/100
2435/2435 [==============================] - 0s 30us/st

In [17]:
score = model.evaluate(testX, testY, batch_size = 32)
print("Test Accuracy: "+"{:.2%}".format(score[1]))

609/609 [==============================] - 0s 85us/step
Test Accuracy: 45.98%


In [18]:
f = open("file.txt", "a")
import sys
np.set_printoptions(threshold=sys.maxsize)
f.write(np.array2string(model.predict(testX)))
f.close()